## 1. 데코레이터 실습

### 1.1 인자없는 함수에 대한 데코레이터

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Before function runs")
        func()
        print("After function runs")
    return wrapper

In [ ]:
@my_decorator
def say_hello():
    print("Hello!")

In [ ]:
say_hello()

### 1.2 인자있는 함수에 대한 데코레이터

In [ ]:
def logger(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} function with {args} {kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned {result}")
        return result
    return wrapper

In [ ]:
@logger
def add(a, b):
    return a + b

In [ ]:
@logger
def greet(name):
    return f"Hello, {name}!"

In [ ]:
add(3, 5)
greet("Dangtong")

# 2. 환경변수 다루기

## 2.1 환경변수 출력

In [ ]:
import os

In [ ]:
for key, value in os.environ.items():
    print(f"{key}: {value}")

# 3. with 문 다루기

## 3.1 파일 읽기 쓰기 예제

In [ ]:
# 파일 쓰기
with open("sample.txt", "w", encoding="utf-8") as f:
    f.write("Hello, world!\n")
    f.write("with 문 실습 예제입니다.")

In [ ]:
# 파일 읽기
with open("sample.txt", "r", encoding="utf-8") as f:
    content = f.read()
    print(content)

> 중요한 것은 with 블록이 끝나면 자동으로 f.close() 실행

> 예외(Exception)이 발생해도 파일이 안전하게 닫힘

## 3.2 with 직접 구현해 보기

In [ ]:
class Demo:
    def __enter__(self):
        print("리소스 열기")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("리소스 정리 (항상 실행)")
        if exc_type:
            print("예외 발생:", exc_value)
            
        return False  # 예외를 다시 propagate

In [ ]:
with Demo():
    print("작업 중…")
    raise ValueError("일부러 예외 발생!")

In [ ]:
import sqlite3

with sqlite3.connect("test.db") as conn:
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS users(name TEXT)")
    cursor.execute("INSERT INTO users VALUES('dangtong')")
    conn.commit()

# 4. WITH VS DECORATOR

In [ ]:
import time

class Timer:   # 시작, 종료 를 명확하게 구현 해야함 (리소스 할당과 정리에 초점)
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, *args):
        print("실행 시간:", time.time() - self.start)

with Timer():
    time.sleep(1)   # 실행 시점에 Timer 실행


In [ ]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print("실행 시간:", time.time() - start)
        return result
    return wrapper

@timer
def do_task(): # 1초 쉬는 함수
    time.sleep(1)

do_task() # 실행 시점에 timer 함수 실행

# 5. 자료구조 변환

## 5.1 dict → 객체

In [ ]:
class User:
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email

In [ ]:
data = {
    "name": "dangtong",
    "age": 42,
    "email": "dangtong@example.com"
}

In [ ]:
# dict → object 변환
user = User(**data)
print(user.name, user.age, user.email)

## 5.2 객체 → dict

In [ ]:
# object → dict
obj_dict = user.__dict__
print(obj_dict)

## 5.3 dict 리스트 → 객체 리스트

In [ ]:
data_list = [
    {"name": "hong", "age": 30, "email": "hong@test.com"},
    {"name": "kim", "age": 25, "email": "kim@test.com"},
]

In [ ]:
users = [User(**item) for item in data_list]

In [ ]:
for u in users:
    print(u.name, u.age)

## 5.4 객체 리스트 → dict 리스트

In [ ]:
obj_list = [u.__dict__ for u in users]
print(obj_list)

## 5.5 dict, json ↔ pydantic Model (실무에서 가장 많이 사용)

In [ ]:
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int
    email: str

In [ ]:

# dict → model
data = {"name": "park", "age": 33, "email": "park@test.com"}
user_object = User(**data)

In [ ]:

# model → dict
print(user_object.model_dump())

In [ ]:
# model → json
print(user_object.model_dump_json())

In [ ]:

# model list
user_object_list = [User(**i) for i in [data, data]]
print([x.model_dump() for x in user_object_list])